# Acquisition de données de poses pour entraînement de modèles

Ce notebook a pour objectif de :
- Initialiser la capture vidéo à partir de la caméra de l’ordinateur.
- Utiliser MediaPipe pour détecter les landmarks (points clés) de la pose humaine.
- Afficher en temps réel le squelette détecté sur l’image.
- Permettre la capture d’une pose particulière (au clavier) pour la stocker.
- Sauvegarder l’ensemble des poses capturées dans un fichier CSV (`poses_capturees.csv`), où chaque ligne contient les coordonnées X et Y des 33 landmarks (66 colonnes).
- Documenter chaque étape en français pour faciliter la compréhension et la réutilisation.

**Instructions d’utilisation :**
1. Cliquez dans la cellule d’acquisition (#5) et exécutez-la.
2. Une fenêtre s’ouvre avec le flux vidéo et le squelette détecté.
3. Appuyez sur la touche **`a`** pour capturer la pose courante (elle sera ajoutée à la liste interne).
4. Appuyez sur **`q`** ou **`Esc`** pour quitter et sauvegarder toutes les poses dans `poses_capturees.csv`.


In [2]:
# ---------------------------------------------------------------
# Imports nécessaires pour la capture vidéo, le traitement MediaPipe,
# l’affichage et la sauvegarde CSV
# ---------------------------------------------------------------

import cv2                              # OpenCV pour la capture webcam et l’affichage
import mediapipe as mp                  # MediaPipe pour la détection de la pose
import csv                              # Pour écrire les coordonnées dans un fichier CSV
import os                               # Pour vérifier ou créer le dossier de sortie

from pose import Pose # Importation de la classe Pose définie dans le fichier pose.py


## Explications et conseils d’utilisation

1. **Initialisation**  
   - Nous avons configuré les paramètres MediaPipe (`MIN_DETECTION_CONFIDENCE`, `MIN_TRACKING_CONFIDENCE`, `MODEL_COMPLEXITY`) pour un compromis entre rapidité et précision.  
   - `VIDEO_SOURCE = 0` correspond à la première caméra disponible. Si vous avez plusieurs caméras, vous pouvez essayer `1`, `2`, etc.

2. **Détection en temps réel**  
   - À chaque itération de la boucle, on lit une image depuis la caméra, on la convertit en RGB puis on fait `pose.process(image_rgb)`.  
   - Si des landmarks sont détectés (`results.pose_landmarks`), on les dessine sur l’image avec `draw_landmarks`.

3. **Capture d’une pose**  
   - Lorsque vous appuyez sur la touche **`a`**, on appelle `pose_tracker._toVector(results.pose_landmarks)` qui renvoie une liste de 66 coordonnées (33 points × (x,y)).  
   - Si aucun landmark n’est détecté, la liste est vide et on ignore cette capture. Sinon, on ajoute la liste à `pose_data`.

4. **Arrêt et sauvegarde**  
   - Lorsque vous appuyez sur **`q`** ou **`Esc`**, on sort de la boucle, on ouvre (ou crée) `poses_capturees.csv` dans le dossier `data_poses/` puis on écrit toutes les lignes de `pose_data`.  
   - Chaque ligne du CSV correspond à une capture (une pose) ; vous retrouverez pour chaque landmark sa coordonnée X (normalisée entre 0 et 1) puis Y (entre 0 et 1).  
   - Le fichier CSV ne contient pas d’en-tête ; si besoin, vous pouvez l’ajouter manuellement (par exemple : `x0,y0,x1,y1,...,x32,y32`).

5. **Variantes et traitements ultérieurs**  
   - **Regroupement par action** : pour chaque action (ex. : `saluer`, `pointer`, `marcher`), vous pouvez dupliquer ce notebook (ou modifier `CSV_FILENAME`) pour générer plusieurs fichiers CSV distincts (ex. : `saluer.csv`, `pointer.csv`, …).  
   - **Labeling** : après la capture, vous pouvez créer un fichier `labels.csv` parallèle, où chaque ligne contient l’étiquette correspondante à chaque entrée de `poses_capturees.csv`.  
   - **Prétraitements supplémentaires** : normalisation supplémentaire, calcul d’angles articulaires, filtration des valeurs aberrantes…  
   - **Entraînement de modèle** : une fois le dataset constitué, vous pourrez charger `pandas.read_csv()` pour préparer vos features et labels, puis entraîner un classifieur (SVM, réseau de neurones, etc.).


In [3]:
# ---------------------------------------------------------------
# Paramètres de détection et de capture
# ---------------------------------------------------------------

MIN_DETECTION_CONFIDENCE = 0.5       # Confiance minimale pour détecter la pose
MIN_TRACKING_CONFIDENCE  = 0.5       # Confiance minimale pour la poursuite
MODEL_COMPLEXITY         = 1         # Complexité du modèle MediaPipe (0, 1 ou 2)
VIDEO_SOURCE             = 0         # Index de la caméra (0 = première caméra)

# Nom du fichier CSV dans lequel on sauvegardera les poses
CSV_FILENAME = "poses_capturees.csv"

# Dossier de sortie (pour s’assurer que le fichier sera créé dans un dossier spécifique)
OUTPUT_DIR = "./data_poses"
os.makedirs(OUTPUT_DIR, exist_ok=True)  # Création du dossier s’il n’existe pas

# Liste qui contiendra toutes les poses capturées (chaque pose = liste de 66 chaînes x,y)
pose_data = []

# Initialisation de l’objet Pose ; la caméra sera ouverte lors de la boucle
pose_tracker = Pose(MIN_DETECTION_CONFIDENCE,
                    MIN_TRACKING_CONFIDENCE,
                    MODEL_COMPLEXITY,
                    VIDEO_SOURCE)

# Initialisation de MediaPipe pour l’affichage des landmarks
mp_drawing = mp.solutions.drawing_utils
mp_pose    = mp.solutions.pose


In [ ]:
# ---------------------------------------------------------------
# Boucle principale : capture vidéo, détection de la pose, affichage,
# capture au clavier, puis sauvegarde CSV à la sortie.
# ---------------------------------------------------------------

# Ouvrir la webcam
cap = cv2.VideoCapture(VIDEO_SOURCE)

# Créer un contexte MediaPipe Pose pour la détection en temps réel
with mp_pose.Pose(min_detection_confidence=MIN_DETECTION_CONFIDENCE,
                  min_tracking_confidence=MIN_TRACKING_CONFIDENCE,
                  model_complexity=MODEL_COMPLEXITY) as pose:

    print("Démarrage de la capture vidéo. Appuyez sur 'a' pour capturer une pose, 'q' ou 'Esc' pour quitter.")
    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            print("Impossible de lire l'image de la caméra.")
            break

        # Miroir horizontal pour plus d’ergonomie
        image = cv2.flip(frame, 1)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Détection de la pose sur l’image RGB
        results = pose.process(image_rgb)

        # Si des landmarks de pose sont trouvés, on les dessine
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                image, 
                results.pose_landmarks, 
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(0,0,255), thickness=2, circle_radius=2)
            )

        # Afficher l’image traitée (avec squelette)
        cv2.imshow("Acquisition de Poses", image)

        # Attendre 5 ms pour la détection d’une touche
        key = cv2.waitKey(5) & 0xFF

        if key == ord('a'):
            # Capturer la pose courante sous forme de vecteur
            vec = pose_tracker._toVector(results.pose_landmarks)
            if vec:
                pose_data.append(vec)
                print(f"Pose capturée ! Nombre total de poses : {len(pose_data)}")
            else:
                print("Aucune pose détectée, rien n'est ajouté.")

        elif key == ord('q') or key == 27:  # 27 = Touche Échap
            print("Quitter et sauvegarder les poses...")
            # Construire le chemin complet du fichier CSV
            chemin_csv = os.path.join(OUTPUT_DIR, CSV_FILENAME)
            with open(chemin_csv, "w", newline="") as f:
                writer = csv.writer(f)
                # Écrire chaque ligne (chaque vecteur de 66 valeurs)
                for row in pose_data:
                    writer.writerow(row)
            print(f"Données sauvegardées dans « {chemin_csv} ». Total de poses : {len(pose_data)}")
            break

    # Libération des ressources
    cap.release()
    cv2.destroyAllWindows()


🔵 Démarrage de la capture vidéo. Appuyez sur 'a' pour capturer une pose, 'q' ou 'Esc' pour quitter.
✔️  Pose capturée ! Nombre total de poses : 1
❗ Quitter et sauvegarder les poses...
✅ Données sauvegardées dans « ./data_poses\poses_capturees.csv ». Total de poses : 1
